# Stage 1: Enhanced Data Cleaning, Preprocessing, and Exploratory Analysis 
In this notebook, we perform **data cleaning, preprocessing, and exploratory analysis (EDA)** on the Cleantech Media and Google Patent datasets. The goal is to identify **trends, key technologies, and innovation gaps** by analyzing media publications and patents.

In [13]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import contractions
import unidecode
from num2words import num2words
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from spellchecker import SpellChecker

## Data Collection and Cleaning
Before analyzing the data, we first **load, inspect, and clean** the datasets:  

- **Load datasets**: We import the **Cleantech Media Dataset** and the **Cleantech Google Patent Dataset** into Pandas DataFrames.  
- **Remove duplicates**: Identical or near-identical entries are removed to prevent data bias.  
- **Handle missing values**: We check for null or incomplete entries and decide whether to impute, replace, or remove them.  
- **Filter relevant information**: Non-informative texts (e.g., generic statements) are removed to ensure high-quality analysis.  

In [2]:
# Load raw data
media_dataset_path = "../data/cleantech_media_dataset_v3_2024-10-28.csv"
google_patent_dataset_path = "../data/cleantech_rag_evaluation_data_2024-09-20.csv"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 30)

# Load CSV files
df_media = pd.read_csv(media_dataset_path, header = 0)
df_google_patents = pd.read_csv(google_patent_dataset_path, sep = ";", header = 0)

print("Raw cleantech_media_dataset_v3_2024-10-28.csv:")
print(df_media.info())
df_media

print("Raw cleantech_rag_evaluation_data_2024-09-20.csv:")
print(df_google_patents.info())
print(df_google_patents.head())

Raw cleantech_media_dataset_v3_2024-10-28.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20111 entries, 0 to 20110
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  20111 non-null  int64  
 1   title       20111 non-null  object 
 2   date        20111 non-null  object 
 3   author      0 non-null      float64
 4   content     20111 non-null  object 
 5   domain      20111 non-null  object 
 6   url         20111 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.1+ MB
None
Raw cleantech_rag_evaluation_data_2024-09-20.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   example_id                  23 non-null     object
 1   question_id                 23 non-null     object
 2   question                    

In [3]:
# Create a new dataframe for the processed data 
df_media_processed = df_media.rename(columns={df_media.columns[0]: 'id'})
df_media_processed.drop(columns=['author'], inplace=True)

# Convert columns to the required data types
df_media_processed['title'] = df_media_processed['title'].astype(str)
df_media_processed['content'] = df_media_processed['content'].astype(str)
df_media_processed['domain'] = df_media_processed['domain'].astype(str)
df_media_processed['url'] = df_media_processed['url'].astype(str)
df_media_processed['date'] = pd.to_datetime(df_media_processed['date'], errors='coerce')
df_media_processed['id'] = df_media_processed['id'].astype(int)

# Check for duplicates
duplicate_ids = df_media_processed[df_media_processed.duplicated(subset=['id'])]
print(duplicate_ids)


df_media_processed

Empty DataFrame
Columns: [id, title, date, content, domain, url]
Index: []


,id,title,date,content,domain,url
0,93320,"XPeng Delivered ~100,000 V...",2022-01-02,['Chinese automotive start...,cleantechnica,https://cleantechnica.com/...
1,93321,Green Hydrogen: Drop In Bu...,2022-01-02,['Sinopec has laid plans t...,cleantechnica,https://cleantechnica.com/...
2,98159,World’ s largest floating ...,2022-01-03,['Huaneng Power Internatio...,pv-magazine,https://www.pv-magazine.co...
3,98158,Iran wants to deploy 10 GW...,2022-01-03,['According to the Iranian...,pv-magazine,https://www.pv-magazine.co...
4,31128,Eastern Interconnection Po...,2022-01-03,['Sign in to get the best ...,naturalgasintel,https://www.naturalgasinte...
...,...,...,...,...,...,...
20106,104263,US Treasury finalises 45X ...,2024-10-24,['The US Department of the...,pv-tech,https://www.pv-tech.org/us...
20107,104264,EDP trials robotic constru...,2024-10-24,['Developer EDP is pilotin...,pv-tech,https://www.pv-tech.org/ed...
20108,101434,Australia has 7.8 GW of ut...,2024-10-24,['The volume of large-scal...,pv-magazine,https://www.pv-magazine.co...
20109,101428,Residential PV prices in G...,2024-10-24,['The comparison site Self...,pv-magazine,https://www.pv-magazine.co...


In [4]:
df_google_patents_processed = df_google_patents.dropna()
df_google_patents_processed = df_google_patents_processed.drop_duplicates(subset=['question_id'])      # Remove duplicate questions
df_google_patents_processed = df_google_patents_processed.rename(columns=lambda x: x.replace(',', '')) # rename column article_url

df_google_patents_processed['example_id'] = df_google_patents_processed['example_id'].astype(int)
df_google_patents_processed['question_id'] = df_google_patents_processed['question_id'].astype(int)
df_google_patents_processed['question'] = df_google_patents_processed['question'].astype(str)
df_google_patents_processed['relevant_text'] = df_google_patents_processed['relevant_text'].astype(str)
df_google_patents_processed['answer'] = df_google_patents_processed['answer'].astype(str)
df_google_patents_processed['article_url'] = df_google_patents_processed['article_url'].astype(str)

df_google_patents_processed


,example_id,question_id,question,relevant_text,answer,article_url
0,1,1,What is the innovation beh...,Leclanché said it has deve...,Leclanché's innovation is ...,https://www.sgvoice.net/st...
1,2,2,What is the EU’s Green Dea...,The Green Deal Industrial ...,The EU’s Green Deal Indust...,https://www.sgvoice.net/po...
3,4,3,What are the four focus ar...,The new plan is fundamenta...,The four focus areas of th...,https://www.sgvoice.net/po...
4,5,4,When did the cooperation b...,What caught our eye was a ...,July 2013,https://cleantechnica.com/...
5,6,5,Did Colgate-Palmolive ente...,"Scout Clean Energy, a Colo...",yes,https://solarindustrymag.c...
6,7,6,What is the status of Zero...,"In December, the US startu...",ZeroAvia's hydrogen fuel c...,https://cleantechnica.com/...
8,9,8,Is Mississipi an anti-ESG ...,Mississippi is among two d...,yes,https://cleantechnica.com/...
9,10,9,Can you hang solar panels ...,Scaling down from the farm...,yes,https://cleantechnica.com/...
10,11,10,Who develops quality contr...,Scientists from the Chines...,Scientists from the Chines...,https://www.azocleantech.c...
11,12,11,Why are milder winters det...,Since grapes and apples ar...,Milder winters are detrime...,https://www.azocleantech.c...


### Analyse dataframe df_media_processed

In [5]:
# Count occurrences of each unique value in the 'domain' column
domain_counts = df_media_processed['domain'].value_counts()

# Display the counts
print(domain_counts)

domain
energy-xprt              4181
pv-magazine              3093
azocleantech             2488
cleantechnica            2089
pv-tech                  1969
thinkgeoenergy           1052
solarpowerportal.co       850
energyvoice               828
solarpowerworldonline     785
solarindustrymag          621
solarquarter              606
rechargenews              573
naturalgasintel           298
iea                       173
energyintel               171
greenprophet              130
greenairnews               59
ecofriend                  55
all-energy                 39
decarbxpo                  20
storagesummit              15
eurosolar                   9
indorenergy                 4
bex-asia                    2
biofuels-news               1
Name: count, dtype: int64


## Text Preprocessing
To ensure that the text data is **ready for NLP tasks**, we preprocess it using common natural language processing (NLP) techniques:  

- **Tokenization**: Split text into individual words or subwords for better analysis.  
- **Stopword Removal**: Common but uninformative words (e.g., "the", "is", "and") are removed.  
- **Lemmatization**: Words are reduced to their root form (e.g., "developing" → "develop").  
- **Lowercasing**: Standardize all text to lowercase to avoid duplicate entries.  

These steps improve the quality of text-based analysis and ensure consistency across datasets.

### df_media

In [17]:
str_columns = ['title', 'content']
PUNCTUATIONS = string.punctuation

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
STOPWORDS = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

spell = SpellChecker()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joshu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joshu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\joshu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\joshu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Following cell takes a while to run

In [25]:
# Lower casing
df_media_processed[str_columns] = df_media_processed[str_columns].apply(lambda x: x.str.lower())

def remove_emails(text):
    return re.sub(r'\S+@\S+', '', text) if isinstance(text, str) else text

def remove_dates(text):
    text = re.sub(r'\d{1,2}(st|nd|rd|th)?[-./]\d{1,2}[-./]\d{2,4}', '', text)
    pattern = re.compile(r'(\d{1,2})?(st|nd|rd|th)?[-./,]?\s?(of)?\s?([J|j]an(uary)?|[F|f]eb(ruary)?|[Mm]ar(ch)?|[Aa]pr(il)?|[Mm]ay|[Jj]un(e)?|[Jj]ul(y)?|[Aa]ug(ust)?|[Ss]ep(tember)?|[Oo]ct(ober)?|[Nn]ov(ember)?|[Dd]ec(ember)?)\s?(\d{1,2})?(st|nd|rd|th)?\s?[-./,]?\s?(\d{2,4})?')
    text = pattern.sub(r'', text)
    return text if isinstance(text, str) else text

def remove_html(text):
    clean_text = BeautifulSoup(text).get_text()
    return clean_text

def remove_tags_mentions(text):
    pattern = re.compile(r'(@\S+|#\S+)')
    return pattern.sub('', text)

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCTUATIONS))

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in STOPWORDS])

def remove_whitespaces(text):
    return " ".join(text.split())

def freq_words(text):
    tokens = word_tokenize(text)
    counter = Counter(tokens)
    FrequentWords = [word for word, _ in counter.most_common(10)]
    return FrequentWords

def remove_fw(text):
    FrequentWords = freq_words(text) 
    tokens = word_tokenize(text)
    without_fw = [word for word in tokens if word not in FrequentWords]
    return ' '.join(without_fw)

def rare_words(text):
    tokens = word_tokenize(text)
    counter = Counter(tokens)
    RareWords = []
    number_rare_words = 10
    for word, word_count in counter.most_common()[-number_rare_words:]:
        RareWords.append(word)
    return RareWords

def remove_rw(text):
    RareWords = rare_words(text)  
    tokens = word_tokenize(text)
    without_rw = [word for word in tokens if word not in RareWords]
    return ' '.join(without_rw)

def nums_to_words(text):                                                                   # needs update so "45x" is also changed to text - tbd JH
    new_text = []
    for word in text.split():
        if word.isdigit() or (word.startswith('-') and word[1:].isdigit()):
            try:
                new_text.append(num2words(word))
            except Exception as e:
                new_text.append(word)
        else:
            new_text.append(word)
    return " ".join(new_text)

# DOES NOT WORK?!?
# def expand_contractions(text):
#     expanded_text = []
#     for line in text:
#         expanded_text.append(contractions.fix(line))
#     return expanded_text

def text_lemmatize(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

def correct_spelling(text):
    if pd.isna(text):  # Check if the value is NaN (None)
        return text  # Return it as is
    
    correct_text = []
    misspelled_words = spell.unknown(text.split())

    for word in text.split():
        if word in misspelled_words:
            corrected_word = spell.correction(word)
            correct_text.append(corrected_word if corrected_word else word)  # Handle None corrections
        else:
            correct_text.append(word)

    return " ".join(correct_text)

def accented_to_ascii(text):
    return unidecode.unidecode(text)

## Call all removals
# df_media_processed[str_columns] = df_media_processed[str_columns].map(remove_emails)
# df_media_processed[str_columns] = df_media_processed[str_columns].map(remove_dates)
# df_media_processed[str_columns] = df_media_processed[str_columns].map(remove_html)
# df_media_processed[str_columns] = df_media_processed[str_columns].map(remove_tags_mentions)
# df_media_processed[str_columns] = df_media_processed[str_columns].map(remove_punctuation)
# df_media_processed[str_columns] = df_media_processed[str_columns].map(remove_stopwords)
# df_media_processed[str_columns] = df_media_processed[str_columns].map(remove_whitespaces)
# df_media_processed['content'] = df_media_processed['content'].map(remove_fw)                    # only do for content, else the title will be removed
# df_media_processed['content'] = df_media_processed['content'].map(remove_rw)  
# df_media_processed[str_columns] = df_media_processed[str_columns].map(nums_to_words)
## df_media_processed[str_columns] = df_media_processed[str_columns].map(expand_contractions)    # DOES NOT WORK AS INTENDED...
# df_media_processed[str_columns] = df_media_processed[str_columns].map(text_lemmatize)
# df_media_processed[str_columns] = df_media_processed[str_columns].map(correct_spelling)        # takes a lot of time!
# df_media_processed[str_columns] = df_media_processed[str_columns].map(accented_to_ascii)

df_media_processed

,id,title,date,content,domain,url
0,93320,xpeng deliv one hundr thou...,2022-01-02,chine startup shown one dr...,cleantechnica,https://cleantechnica.com/...
1,93321,green hydrogen drop bucket...,2022-01-02,laid plan build largest pr...,cleantechnica,https://cleantechnica.com/...
2,98159,world' largest float pv pl...,2022-01-03,intern switch three hundr ...,pv-magazine,https://www.pv-magazine.co...
3,98158,iran want deploy ten gw re...,2022-01-03,iranian author current 80g...,pv-magazine,https://www.pv-magazine.co...
4,31128,eastern interconnect power...,2022-01-03,sign get best natur news d...,naturalgasintel,https://www.naturalgasinte...
...,...,...,...,...,...,...
20106,104263,u treasuri finalis 45x adv...,2024-10-24,depart treasuri dot finali...,pv-tech,https://www.pv-tech.org/us...
20107,104264,edp trial robot construct ...,2024-10-24,develop pilot robot solut ...,pv-tech,https://www.pv-tech.org/ed...
20108,101434,australia seventy-eight gw...,2024-10-24,volum largescal project co...,pv-magazine,https://www.pv-magazine.co...
20109,101428,residenti pv price germani...,2024-10-24,comparison site selfmad en...,pv-magazine,https://www.pv-magazine.co...


To save some time, we save the processed/cleaned dataframe to directly load it for further steps:

In [26]:
df_media_processed.to_csv('../cleaned_data/media_dataset_cleaned.csv', index=False)

### df_google_patents

In [27]:
str_columns = ['question', 'relevant_text', 'answer', 'article_url']
df_google_patents_processed[str_columns] = df_google_patents_processed[str_columns].apply(lambda x: x.str.lower())

df_google_patents_processed

,example_id,question_id,question,relevant_text,answer,article_url
0,1,1,what is the innovation beh...,leclanché said it has deve...,leclanché's innovation is ...,https://www.sgvoice.net/st...
1,2,2,what is the eu’s green dea...,the green deal industrial ...,the eu’s green deal indust...,https://www.sgvoice.net/po...
3,4,3,what are the four focus ar...,the new plan is fundamenta...,the four focus areas of th...,https://www.sgvoice.net/po...
4,5,4,when did the cooperation b...,what caught our eye was a ...,july 2013,https://cleantechnica.com/...
5,6,5,did colgate-palmolive ente...,"scout clean energy, a colo...",yes,https://solarindustrymag.c...
6,7,6,what is the status of zero...,"in december, the us startu...",zeroavia's hydrogen fuel c...,https://cleantechnica.com/...
8,9,8,is mississipi an anti-esg ...,mississippi is among two d...,yes,https://cleantechnica.com/...
9,10,9,can you hang solar panels ...,scaling down from the farm...,yes,https://cleantechnica.com/...
10,11,10,who develops quality contr...,scientists from the chines...,scientists from the chines...,https://www.azocleantech.c...
11,12,11,why are milder winters det...,since grapes and apples ar...,milder winters are detrime...,https://www.azocleantech.c...


## Exploratory Data Analysis
EDA helps us **understand data patterns and distributions** before applying complex NLP models. We perform:  

- **Temporal Analysis**: We examine **publication trends** over time to detect emerging Cleantech topics.  
- **Named Entity Recognition (NER)**: Identify key **companies, organizations, and technologies** frequently mentioned in the datasets.  
- **Word Frequency Analysis**: Find the most common words and phrases across media and patents.  
- **Visualization**:  
  - **Word Clouds** to showcase frequently occurring terms  
  - **Bar Charts** to compare key industry players and technology mentions  
  - **Network Graphs** to analyze relationships between companies and technologies  

## Topic Modeling
To **identify hidden themes and emerging trends**, we apply topic modeling techniques on both datasets:  

- **Latent Dirichlet Allocation (LDA)** and **Non-Negative Matrix Factorization (NMF)** to uncover broad thematic structures.  
- **Top2Vec** and **BERTopic** for **more dynamic and context-aware topic modeling**.  
- **Comparing Media vs. Patents**:  
  - Which Cleantech topics are **gaining media attention** but **not patented** yet?  
  - Are **patents aligned with market trends**, or do they focus on different areas?  
  - **What are the innovation gaps** between research and real-world applications?  

By the end of this step, we will have a **structured view of the Cleantech landscape**, highlighting **key trends, players, and technological opportunities**.
